In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

**Website Scrapper Justia**

In [ ]:

def process_subtitle(soup, title_num, subtitle_url):

    data = []


    section_pattern = f'/title-{title_num}/subtitle-\d+/section-{title_num}-\d+(?:-\d+)?/?$'
    section_data = find_links(soup, section_pattern, title_num)

    if section_data:

        for link_info in section_data:
            entry = {
                'Title': title_num,
                'Section': re.sub(r'^(\d+)-\d+-', r'\1-', link_info['section_number']),
                'Description': link_info['description'],
                'Path': 'Direct'
            }
            data.append(entry)
            print(f"Added subtitle section entry: {entry}")
            time.sleep(1)
    else:

        roman_numerals = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x']
        for numeral in roman_numerals:
            part_pattern = f'/title-{title_num}/subtitle-\d+/part-{numeral}/?$'
            part_links = soup.find_all('a', href=re.compile(part_pattern, re.IGNORECASE))

            for part_link in part_links:
                part_url = build_url(part_link['href'])
                print(f"\nProcessing roman numeral part: {part_url}")

                part_soup = fetch_page(part_url)
                if not part_soup:
                    continue


                part_section_pattern = f'/title-{title_num}/subtitle-\d+/part-{numeral}/section-{title_num}-\d+(?:-\d+)?/?$'
                part_section_data = find_links(part_soup, part_section_pattern, title_num)

                for link_info in part_section_data:
                    entry = {
                        'Title': title_num,
                        'Section': re.sub(r'^(\d+)-\d+-', r'\1-', link_info['section_number']),
                        'Description': link_info['description'],
                        'Path': f'Part {part_link.get_text(strip=True)}'
                    }
                    data.append(entry)
                    print(f"Added roman numeral part entry: {entry}")
                    time.sleep(1)

    return data

def process_title(title_num, title_url):
    """Process a single title and return its data"""
    data = []
    print(f"\nProcessing Title {title_num}: {title_url}")

    title_soup = fetch_page(title_url)
    if not title_soup:
        return data


    direct_section_pattern = f'/title-{title_num}/section-{title_num}-\d+(?:-\d+)?/?$'
    direct_section_data = find_links(title_soup, direct_section_pattern, title_num)

    for link_info in direct_section_data:
        entry = {
            'Title': title_num,
            'Section': re.sub(r'^(\d+)-\d+-', r'\1-', link_info['section_number']),
            'Description': link_info['description'],
            'Path': 'Direct'
        }
        data.append(entry)
        print(f"Added direct entry: {entry}")
        time.sleep(1)


    subtitle_pattern = f'/title-{title_num}/subtitle-\d+/?$'
    subtitle_links = title_soup.find_all('a', href=re.compile(subtitle_pattern))

    for subtitle_link in subtitle_links:
        subtitle_url = build_url(subtitle_link['href'])
        subtitle_number = subtitle_link['href'].split('subtitle-')[-1].strip('/')

        print(f"\nProcessing subtitle: {subtitle_url}")

        subtitle_soup = fetch_page(subtitle_url)
        if not subtitle_soup:
            continue


        subtitle_data = process_subtitle(subtitle_soup, title_num, subtitle_url)
        data.extend(subtitle_data)

    return data



def natural_sort_key(section):
    """Create a tuple key for natural sorting of section numbers"""
    parts = re.split(r'[-.]', section)
    return tuple(int(part) if part.isdigit() else part for part in parts)

def main():
    all_data = []

    titles = get_all_titles()

    print(f"Found {len(titles)} titles to process")

    for title_num, title_url in titles:
        title_data = process_title(title_num, title_url)

        all_data.extend(title_data)


        if all_data:
            df = pd.DataFrame(all_data)


            df['Title'] = pd.to_numeric(df['Title'])


            df.sort_values(['Title', 'Section'],
                           key=lambda x: x.map(lambda y: natural_sort_key(str(y)) if pd.notnull(y) else None),
                           inplace=True)

            df.to_csv('maryland_transportation_all_titles.csv', index=False)

            print(f"\nIntermediate save: {len(df)} total sections processed")


    if all_data:
        print("\nScraping complete. Final dataset:")

        df_final = pd.DataFrame(all_data)

        df_final.sort_values(['Title', 'Section'],
                             key=lambda x: x.map(lambda y: natural_sort_key(str(y)) if pd.notnull(y) else None),
                             inplace=True)

        df_final.to_csv('maryland_transportation_all_titles.csv', index=False)  # Save final DataFrame

        print(df_final.head())

        print(f"\nTotal sections: {len(df_final)}")

if __name__ == "__main__":
    main()


Found 27 titles to process

Processing Title 1: https://law.justia.com/codes/maryland/transportation/title-1
Found 3 links matching pattern: /title-1/section-1-\d+(?:-\d+)?/?$
Link found: /codes/maryland/transportation/title-1/section-1-101/ | Section: 1-1-101 | Description: Definitions
Link found: /codes/maryland/transportation/title-1/section-1-102/ | Section: 1-1-102 | Description: Certificate of Compliance With Workers' Compensation Act
Link found: /codes/maryland/transportation/title-1/section-1-103/ | Section: 1-1-103 | Description: Renewal of License or Permit; Verification of Payment of Taxes and Unemployment Insurance Contributions
Added direct entry: {'Title': '1', 'Section': '1-101', 'Description': 'Definitions', 'Path': 'Direct'}
Added direct entry: {'Title': '1', 'Section': '1-102', 'Description': "Certificate of Compliance With Workers' Compensation Act", 'Path': 'Direct'}
Added direct entry: {'Title': '1', 'Section': '1-103', 'Description': 'Renewal of License or Permit;

**Dataset Loading and Dropping any null instances for Driver State**

In [ ]:
df = pd.read_csv('Traffic_Violations_20241223.csv')
df = df.dropna(subset=['Driver State'])



<ipython-input-9-b34f07558136>:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Traffic_Violations_20241223.csv')


**Stratified Sampling**

In [ ]:
total_points = 100000
frac_per_state = total_points / len(df)

stratified_sample = df.groupby('Driver State', group_keys=False).apply(
    lambda x: x.sample(frac=frac_per_state, random_state=30)
)


stratified_sample.reset_index(drop=True, inplace=True)


stratified_sample.to_csv('Sampled_Traffic_Violations.csv', index=False)

<ipython-input-28-f8f5cc314790>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby('Driver State', group_keys=False).apply(


**Creating Charge_Abbreviation Attribute**

In [ ]:

us_state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
    'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
    'VA', 'WA', 'WV', 'WI', 'WY',
]

df['Charge_Abbreviation'] = df['Charge'].str[:8]




2
Presence and count of each U.S. state abbreviation in the dataset:
Nothing: 13
Contraband Only: 7


In [ ]:
df = pd.read_csv('Sampled_Traffic_Violations.csv')




In [ ]:

df['Charge_Abbreviation'] = df['Charge'].str.extract(r'^([^(]*)').fillna('')


df['Charge_Abbreviation'] = df['Charge_Abbreviation'].str.strip()



Index(['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Driver City',
       'Driver State', 'DL State', 'Arrest Type', 'Geolocation',
       'Charge_Abbreviation'],
      dtype='object')
Total unique charges: 5

Presence and count of each charge abbreviation in the dataset:
WHITE: 226
BLACK: 226
HISPANIC: 168
ASIAN: 70
OTHER: 70


**Adding Unknown values for Search Conducted and Search Outcome attribute**

In [ ]:

df['Search Conducted'] = df['Search Conducted'].fillna('Unknown')


df.loc[df['Search Conducted'] == 'Unknown', 'Search Outcome'] = 'Unknown'
df.loc[df['Search Conducted'] == 'Unknown', 'Search Reason For Stop'] = 'Unknown'



state_filter_df = df[df.State.isin(us_state_abbreviations)]


state_filter_df.to_csv('Sampled_Traffic_Violations.csv', index=False)


In [ ]:
df1 = pd.read_csv('maryland_transportation_all_titles.csv')
df1 = df1.drop(columns=['Title', 'Path'])



**Merging and Dropping**

In [ ]:
merged_df = pd.merge(df, df1, left_on='Charge_Abbreviation', right_on='Section', how='left')

merged_df = merged_df.drop(columns=['Geolocation'])
merged_df.to_csv('maryland_traffic_violations.csv', index=False)

In [ ]:

df2 = pd.read_csv('ESERO Code.csv')

**Merging**

In [ ]:

merged_df = pd.merge(merged_df, df2, left_on='Charge_Abbreviation', right_on='Number', how='left')


merged_df['Charge Hierarchy'] = merged_df['Charge_Abbreviation'].replace('', None).fillna(merged_df['Number'])


merged_df['Charge Description'] = merged_df['Description_y'].replace('', None).fillna(merged_df.get('Description_yy'))


In [ ]:
merged_df = merged_df.drop(columns=['Charge_Abbreviation', 'Description_y', 'Description_yy', 'Number', 'Section', 'Article'])

In [ ]:
merged_df = merged_df[merged_df['Latitude'] != 0]

**Unique Instances**

In [ ]:
df1 = pd.read_csv('Transcode1.csv')
df2 = pd.read_csv('Transcode2.csv')


combined_df = pd.concat([df1, df2])


def create_charge(row):
    title = int(row['Title'])


    section = float(row['Section'])

    if section.is_integer():
        section_str = str(int(section))
    else:
        section_str = str(section)

    if pd.isna(row['Subsection']) or str(row['Subsection']).strip() == '':
        return f"{title}-{section_str}"
    else:
        return f"{title}-{section_str}({row['Subsection']})"


def clean_description(text):
    if pd.isna(text):
        return text

    parts = text.split('..')

    cleaned = parts[0].strip()
    return cleaned

combined_df['Charge'] = combined_df.apply(create_charge, axis=1)


combined_df['Description'] = combined_df['Description'].apply(clean_description)

s
unique_df = combined_df.drop_duplicates(subset='Charge')
unique_df = unique_df[['Charge'] + [col for col in unique_df.columns if col != 'Charge']]
unique_df = unique_df.drop(columns=['Title', 'Section', 'Subsection'])


unique_df['Fine Description'] = unique_df['Description'].str.extract(r'^([A-Za-z0-9\s]+)')[0].str.strip()

In [ ]:
merged_df = pd.merge(merged_df, unique_df, left_on='Charge', right_on='Charge', how='left')

**Adding Additional Fine Values**

In [ ]:

def get_fine_details(speed):
    if speed >= 1 and speed <= 9:
        return {'Fine': '$80.00', 'Points': 1, 'Contr.Acc Fine': '$120.00', 'Contr.Acc Points': 3}
    elif speed >= 10 and speed <= 19:
        return {'Fine': '$90.00', 'Points': 2, 'Contr.Acc Fine': '$130.00', 'Contr.Acc Points': 3}
    elif speed >= 20 and speed <= 29:
        return {'Fine': '$160.00', 'Points': 2, 'Contr.Acc Fine': '$200.00', 'Contr.Acc Points': 3}
    elif speed >= 30 and speed <= 39:
        return {'Fine': '$290.00', 'Points': 5, 'Contr.Acc Fine': '$290.00', 'Contr.Acc Points': 5}

    elif speed >= 40:
        return {'Fine': '$530.00', 'Points': 5, 'Contr.Acc Fine': '$530.00', 'Contr.Acc Points': 5}
    else:
        return {'Fine': '$0.00', 'Points': 0, 'Contr.Acc Fine': '$0.00', 'Contr.Acc Points': 0}


def extract_speed(description):
    match = re.search(r'(\d{2,3})\s*MPH', description)
    if match:
        return int(match.group(1))
    return None


def update_fine_details(row):

    if row['Charge'] == '21-801.1':
        speed = extract_speed(row['Description_x'])
        if speed is not None:
            fine_details = get_fine_details(speed)
            row['Fine'] = fine_details['Fine']
            row['Points'] = fine_details['Points']
            row['Contr.Acc Fine'] = fine_details['Contr.Acc Fine']
            row['Contr.Acc Points'] = fine_details['Contr.Acc Points']


    if row['Charge'] == '16-112(c)':
        row['Fine'] = '$50.00'
        row['Points'] = 0
        row['Contr.Acc Fine'] = '$50.00'
        row['Contr.Acc Points'] = 0
        row['Description'] = 'Failure of individual driving on hwy. to display lic. to uniformed police on demand'


    if row['Charge'] == '21-1124.2(d2)':
        row['Fine'] = '$83.00'
        row['Points'] = 0
        row['Contr.Acc Fine'] = '$83.00'
        row['Contr.Acc Points'] = 0
        row['Description'] = 'Driver using hands to use handheld telephone while motor vehicle is in motion'


    if row['Charge'] == '16-303(c)':
        row['Fine'] = 'MA'
        row['Points'] = 12
        row['Contr.Acc Fine'] = 'MA'
        row['Contr.Acc Points'] = 12
        row['Description'] = 'Driver using hands to use handheld telephone while motor vehicle is in motion'

    if row['Charge'] == '16-303(h)':
        row['Fine'] = 'MA'
        row['Points'] = 12
        row['Contr.Acc Fine'] = 'MA'
        row['Contr.Acc Points'] = 12
        row['Description'] = 'Driving motor veh. while lic. suspended under TR'


    if row['Charge'] == '22-412.3(b)':
        row['Fine'] = '$83.00'
        row['Points'] = 0
        row['Contr.Acc Fine'] = '$83.00'
        row['Contr.Acc Points'] = 0
        row['Description'] = 'Oper. motor veh. with (operator, occupant under age 16) not restrained by (seat belt, child safety seat)'


    if row['Charge'] == '16-101(a)' or row['Charge'] == '16-101(a1)':
        row['Fine'] = 'MA'
        row['Points'] = 5
        row['Contr.Acc Fine'] = 'MA'
        row['Contr.Acc Points'] = 5
        row['Description'] = 'Driving, Attempting to drive) motor veh. on hwy. without req. license and authorization'


    if row['Charge'] == '21-901.1(b)':
        row['Fine'] = '$240.00'
        row['Points'] = 1
        row['Contr.Acc Fine'] = '$280.00'
        row['Contr.Acc Points'] = 3
        row['Description'] = 'Negligent driving veh. in careless and imprudent manner endangering property, life, and person'


    if row['Charge'] == '22-201.1':
        row['Fine'] = '$60.00'
        row['Points'] = 00
        row['Contr.Acc Fine'] = '$60.00'
        row['Contr.Acc Points'] = 00
        row['Description'] = 'Failure of veh. on hwy. to display (lighted lamps, illuminating devices) in unfavorable visibility conditions'

    if row['Charge'] == '21-201.1':
        row['Fine'] = '$90.00'
        row['Points'] = 1
        row['Contr.Acc Fine'] = '$130.00'
        row['Contr.Acc Points'] = 3
        row['Description'] = 'Driver failure to obey properly placed traffic control device instructions'




    if row['Charge'] == '22-219(a)':
        row['Fine'] = '$70.00'
        row['Points'] = 0
        row['Contr.Acc Fine'] = '$70.00'
        row['Contr.Acc Points'] = 0
        row['Description'] = 'Failure to equip veh. with required rear red stop lamp visible from a distance of not less than 300 ft to the rear in normal sunlight'


    if row['Charge'] == '23-104':
        row['Fine'] = '$70.00'
        row['Points'] = 0
        row['Contr.Acc Fine'] = '$70.00'
        row['Contr.Acc Points'] = 0
        row['Description'] = 'Driving (motor veh., autocycle, low speed veh.) not in accordance with established equipment, performance, and other technical standards'
    return row

merged_df = merged_df.apply(update_fine_details, axis=1)




**Merging Fine and Charge**

In [ ]:
merged_df = merged_df[~((merged_df['Fine'].isna()) & (merged_df['Charge'].str.contains('-', na=False)))]




**Generating Section Titles**

In [ ]:



data = {
    "Charge": ["21-1201", "22-101", "24-111", "13-101", "14-110", "26-101", "26-408.1"],
}
df = pd.DataFrame(data)


section_ranges = {
    "Motor Vehicle Administration": ("12-101", "12-209"),
    "Certificates of Title and Registration of Vehicles": ("13-101", "13-955"),
    "Anti-Theft Laws and Related Provisions": ("14-101", "14-110"),
    "For Rental Vehicles": ("15-201", "15-213"),
    "Licensing of Businesses and Occupations": ("15-101", "15-704"),
    "Drivers' Licenses": ("16-101", "16-819"),
    "Financial Responsibility and Vehicle Registration": ("17-101", "17-110"),
    "Civil Liability of Governmental Agencies": ("18-101", "18-106"),
    "Accidents and Accident Reports": ("20-101", "20-110"),
    "Rules of Road": ("21-101", "21-708"),
    "Speed Restrictions": ("21-800", "21-806"),
    "Driving Offenses": ("21-901.1", "21-902"),
    "Parking Violations": ("21-1001", "21-1007"),
    "Miscellaneous Rules": ("21-1101", "21-1119"),
    "Operation of Bicycles and Play Vehicles": ("21-1201", "21-1213"),
    "Operation of Motorcycles": ("21-1301", "21-1307"),
    "Operation of Vehicles on Certain Toll Facilities": ("21-1401", "21-1411"),
    "Equipment of Vehicles": ("22-101", "22-610"),
    "Inspection of Used Vehicles and Warnings for Defective Equipment": ("23-101", "23-208"),
    "Size, Weight, and Load; Highway Preservation": ("24-101", "24-111"),
    "Respective Powers of State and Local Authorities": ("25-101", "25-210"),
    "Parties and Procedures on Citation, Arrest, Trial, and Appeal": ("26-101", "26-408.1")
}


def map_to_section(charge):
    for section, (start, end) in section_ranges.items():
        if start <= charge <= end:
            return section
    return "Unknown Section"


merged_df["Section"] = merged_df["Charge"].apply(map_to_section)



,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description_x,Location,Latitude,Longitude,Accident,...,Arrest Type,Charge Hierarchy,Charge Description,Description,Fine,Points,Contr.Acc Fine,Contr.Acc Points,Fine Description,Section
0,35333459-ff22-4f0a-b69e-a7ad633be9c9,2025-01-02,19:43:00,MCP,"6th District, Gaithersburg / Montgomery Village",KNOWINGLY DRIVING UNINSURED VEHICLE,SAM EIG HWY @ FIELDS RD,39.115861,-77.208437,No,...,A - Marked Patrol,17-107,NaN,Knowingly driving uninsured vehicle,MA,5.0,MA,5.0,Knowingly driving uninsured vehicle,Financial Responsibility and Vehicle Registration
1,fb58b70b-58cc-44b9-b81a-6971bd89e3c6,2025-01-01,04:51:00,MCP,"3rd District, Silver Spring",FAILURE OF VEH. ON HWY. TO DISPLAY LIGHTED LAM...,COLESVILLE RD @ HASTINGS DR,39.013269,-77.016302,No,...,A - Marked Patrol,22-201.1,When Lighted Lamps Are Required,Failure of veh. on hwy. to display (lighted la...,$60.00,0.0,$60.00,0.0,NaN,Equipment of Vehicles
2,770d0b11-a4cf-4cc2-b821-047a54875fe9,2025-01-02,11:18:00,MCP,Headquarters and Special Operations,WINDOW TINT,RANDOLOPH / HUNTERS LANE,39.053935,-77.101225,No,...,A - Marked Patrol,61,TINT,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section
3,c3ee4476-6e6b-4a33-a0d7-a9beafd43e9f,2025-01-03,12:50:00,MCP,Headquarters and Special Operations,WINDOW TINT,E GUDE / DOVER,39.098282,-77.137428,No,...,A - Marked Patrol,61,TINT,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section
4,25afec33-dbc5-4a13-abc9-eb345a7c856e,2025-01-03,07:39:00,MCP,Headquarters and Special Operations,EXCEEDING THE POSTED SPEED LIMIT OF 40 MPH,RANDOLPH RD/TIMBER VIEW CT,39.184407,-77.263585,No,...,A - Marked Patrol,21-801.1,Maximum Limits,NaN,$530.00,5.0,$530.00,5.0,NaN,Speed Restrictions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,512cfa88-42cb-42df-919a-b8d15db9700e,2025-01-02,10:47:00,MCP,Headquarters and Special Operations,WINDOW TINT,220 RANDOLPH RD,39.069464,-77.014071,No,...,L - Motorcycle,61,TINT,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section
630,9b29c1d1-b3ee-4a21-a32c-976397006dd2,2025-01-02,10:34:00,MCP,Headquarters and Special Operations,WINDOW TINT,310 RANDOLPH RD,39.069078,-77.014510,No,...,L - Motorcycle,61,TINT,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section
631,1267d828-7739-47ef-b840-e7c68b15e29f,2025-01-01,22:32:00,MCP,"5th District, Germantown",TAILLIGHTS (*),CLOOPER @ ALLSPICE,39.150045,-77.244195,No,...,A - Marked Patrol,56*,Taillights,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section
632,57e7538d-12ec-4814-a738-324b7d4fcf90,2025-01-02,20:09:00,MCP,"5th District, Germantown",STOP LIGHTS (*),MIDDLEBROOK @ FATHER HURLEY,39.182712,-77.274797,No,...,A - Marked Patrol,64*,Stop Lights,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Section


**Dropping Attributes**

In [ ]:
new_df = merged_df.drop(columns=[ 'Description', 'Fine Description'])
new_df = new_df.rename(columns={ 'Description_x': 'Description'})

In [ ]:
new_df.to_csv('Maryland_Traffic_Violation_2025.csv', index=False)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
from requests.exceptions import ReadTimeout, ConnectTimeout
import logging
from tenacity import retry, stop_after_attempt, wait_exponential

**Generating Driver_City_Latitude and Driver_City_Longitude based on Driver City Attribute**

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def add_city_coordinates(df):

    geolocator = Nominatim(user_agent="my_traffic_app", timeout=10)


    city_coords_cache = {}

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=4, max=10),
        retry_error_callback=lambda _: (None, None)
    )
    def get_coordinates_with_retry(city_name):

        location = geolocator.geocode(city_name)
        if location:
            return location.latitude, location.longitude
        return None, None

    def get_coordinates(city):

        if pd.isna(city):
            return None, None

        city = str(city).strip().upper()
        search_term = f"{city}, USA"

        if search_term in city_coords_cache:
            return city_coords_cache[search_term]

        try:
            coords = get_coordinates_with_retry(search_term)
            if coords != (None, None):
                city_coords_cache[search_term] = coords
            return coords

        except (GeocoderTimedOut, GeocoderUnavailable, ReadTimeout, ConnectTimeout) as e:
            logger.warning(f"Geocoding failed for {city}: {str(e)}")
            return None, None
        except Exception as e:
            logger.error(f"Unexpected error for {city}: {str(e)}")
            return None, None


    unique_cities = df['Driver City'].dropna().unique()
    total_cities = len(unique_cities)
    logger.info(f"Processing {total_cities} unique cities...")


    city_coords = {}
    for i, city in enumerate(unique_cities, 1):
        lat, lon = get_coordinates(city)
        city_coords[city] = {'lat': lat, 'lon': lon}


        if i % 10 == 0:
            logger.info(f"Processed {i}/{total_cities} cities ({(i/total_cities*100):.1f}%)")


        time.sleep(0.5)

    df['Driver_City_Latitude'] = df['Driver City'].map(lambda x: city_coords.get(x, {}).get('lat'))
    df['Driver_City_Longitude'] = df['Driver City'].map(lambda x: city_coords.get(x, {}).get('lon'))


    total_records = len(df['Driver City'].dropna())
    records_with_coords = len(df[df['Driver_City_Latitude'].notna()])
    logger.info("\nGeocoding Summary:")
    logger.info(f"Total cities processed: {total_records}")
    logger.info(f"Cities with coordinates: {records_with_coords}")
    logger.info(f"Success rate: {(records_with_coords/total_records*100):.1f}%")


    cache_df = pd.DataFrame.from_dict(
        {k: [v['lat'], v['lon']] for k, v in city_coords.items()},
        orient='index',
        columns=['latitude', 'longitude']
    )
    cache_df.to_csv('city_coordinates_cache.csv')

    return df
df = pd.read_csv('Maryland_Traffic_Violation_2025.csv')
df = add_city_coordinates(df)


In [ ]:
df.to_csv('Maryland_Traffic_Violation_2025.csv', index=False)